<a href="https://colab.research.google.com/github/hshen13/workable-colab/blob/main/opensora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Conda 的安装：


In [ ]:
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh


In [ ]:
source ~/miniconda3/bin/activate


Opensora的安装

In [ ]:
# create a virtual env and activate (conda as an example)
conda create -n opensora python=3.10
conda activate opensora


#如果找不到opensora的虚拟环境，再安装一次conda

# download the repo
git clone https://github.com/hpcaitech/Open-Sora
cd Open-Sora

# Ensure torch >= 2.4.0
#pip install -v . # for development mode, `pip install -v -e .`
pip install -v -e .
pip install xformers==0.0.27.post2 --index-url https://download.pytorch.org/whl/cu121 # install xformers according to your cuda version
pip install flash-attn --no-build-isolation


模型下载：

In [ ]:
pip install "huggingface_hub[cli]"
huggingface-cli download hpcai-tech/Open-Sora-v2 --local-dir ./ckpts


继续安装其他依赖：

In [ ]:
pip install pandarallel # for parallel processing


In [ ]:
pip install cmake
git clone https://github.com/hpcaitech/TensorNVMe.git && cd TensorNVMe
pip install -r requirements.txt
DISABLE_AIO=1 pip install -v --no-cache-dir .
DISABLE_URING=1 pip install -v --no-cache-dir .


conda install -c conda-forge libstdcxx-ng
conda update libstdcxx-ng

sudo apt-get update
sudo apt-get install libaio1



tensornvme check


In [ ]:
 cd ..


推理：

In [ ]:
torchrun --master_addr localhost --master_port 29500 --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea"

In [ ]:
torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea"

注意ubuntu主机地址的名称
https://blog.csdn.net/lin_xiao_yi/article/details/132490694

easyanimate可以lora，可以生成。



修改cache dir

t5 = dict(
    type="text_embedder",
    from_pretrained="google/t5-v1_1-xxl",
    cache_dir="/mnt/ddn/sora/tmp_load/huggingface/hub/",
    max_length=512,
    shardformer=True,
)
clip = dict(
    type="text_embedder",
    from_pretrained="openai/clip-vit-large-patch14",
    cache_dir="/mnt/ddn/sora/tmp_load/huggingface/hub/",
    max_length=77,
)


训练：

In [ ]:
#训练diffusion的脚本
torchrun --nproc_per_node 1 scripts/diffusion/train.py configs/diffusion/train/stage1.py --d
ataset.data-path datasets/pexels_45k_necessary.csv


In [ ]:
#训练vae的脚本
torchrun --nproc_per_node 1 scripts/vae/train.py configs/vae/train/video_dc_ae_disc.py --dataset.data-path datasets/pexels_45k_necessary.csv

可以用我们写的脚本 data_scencetransition_opensora.ipynb生成符合opensora格式要求的csv
https://colab.research.google.com/drive/1cC874FG1u6IVGlIiax6At5B9mug7kAts?usp=sharing



我们的脚本DataPrepare_VideoWithScenceTransition.ipynb可以下载panda70m数据，切割出带有一个转场的10秒片段，text打标（对一个视频片段画面场景的文字描述）。


https://colab.research.google.com/drive/1i_PO4Xw1Ke0EG8Y-mmOXj3EYN6mc5Ztj
这个脚本生成的json文件格式为：
/content/json_of_internal_datasets.json。 该json文件的格式如下：
 { "file_path": "train/-1hTK0wGrTg_0-5.mp4",
 "text": "previous scene: a green neon sign that reads ' glo '; then next scence: a green logo for a company",
"type": "video" },






torchrun --nproc_per_node 1 --standalone scripts/vae/inference.py configs/vae/inference/video_dc_ae.py --save-dir samples/dcae  --model.from_pretrained outputs/250324_104951-vae_train_video_dc_ae_disc


数据集使用rclone 同步
# https://rclone.org/install/


In [ ]:
rclone copy workspace gdrive:backup
rclone copy workspace gdrive:newfolder --progress

rmdir /home/ubuntu/internvl2_model/

df -h


rclone copy --max-age 24h --progress --no-traverse /path/to/src remote:
zip -r /home/ubuntu/workspace/Colab_Video_Processing/video_caption.zip /home/ubuntu/workspace/Colab_Video_Processing/video_caption


RTC:workspace/Colab_Video_Processing/video_caption

rclone copy /home/ubuntu/workspace/Colab_Video_Processing/video_caption.zip gdrive:newfolder/Colab_Video_Processing/video_caption.zip --progress


rclone copy /home/ubuntu/workspace/Colab_Video_Processing/video_caption.zip gdrive:newfolder/Colab_Video_Processing/video_caption.zip --progress

In [ ]:
torchrun --nproc_per_node 1 scripts/vae/train.py configs/vae/train/video_dc_ae_disc.py  --dataset.data-path datasets/video_analysis_result.csv
